# Aggregate weather conditions based on existent data (past 4 years)

In [1]:
import pandas as pd
import numpy as np

pd.set_option('max_columns', None)
pd.set_option('display.max_rows', 10000)

In [2]:
df = pd.read_csv('data/cleaned_data.csv')
print(df.shape)
df.head()

(609791, 21)


,City,State,Wind Direction,Latitude,Cloud Cover (%),Minimum Temperature (degF),Date time,Precipitation (in),Dew Point (degF),Relative Humidity (%),Precipitation Cover (%),Longitude,Temperature (degF),Maximum Temperature (degF),Visibility (mi),Wind Speed (mph),Snow Depth (in),Sea Level Pressure (mb),Snow (in),Wind Gust (mph),Conditions
0,Akron,OH,180.46,41.08431,10.3,23.2,2017-01-01,0.00,23.3,66.47,0.00,-81.51431,33.7,42.2,9.5,10.5,0.41,1020.8,0.00,26.420465,Clear
1,Akron,OH,84.79,41.08431,82.1,33.0,2017-01-02,0.00,37.6,85.22,16.67,-81.51431,41.8,47.8,3.4,7.3,0.00,1020.3,0.00,26.420465,Overcast
2,Akron,OH,179.42,41.08431,90.3,42.0,2017-01-03,0.37,43.6,90.06,66.67,-81.51431,46.3,48.8,5.0,13.1,0.00,1005.3,0.00,29.800000,"Rain, Overcast"
3,Akron,OH,275.25,41.08431,95.8,20.8,2017-01-04,0.02,20.8,68.33,20.83,-81.51431,30.2,43.0,8.6,31.5,0.00,1008.3,0.00,39.600000,"Rain, Overcast"
4,Akron,OH,257.54,41.08431,74.0,12.6,2017-01-05,0.02,10.7,71.11,37.50,-81.51431,18.5,20.8,5.2,15.1,1.02,1015.0,1.02,24.200000,"Snow, Partially cloudy"


In [4]:
df.dtypes

City                           object
State                          object
Wind Direction                float64
Latitude                      float64
Cloud Cover (%)               float64
Minimum Temperature (degF)    float64
Date time                      object
Precipitation (in)            float64
Dew Point (degF)              float64
Relative Humidity (%)         float64
Precipitation Cover (%)       float64
Longitude                     float64
Temperature (degF)            float64
Maximum Temperature (degF)    float64
Visibility (mi)               float64
Wind Speed (mph)              float64
Snow Depth (in)               float64
Sea Level Pressure (mb)       float64
Snow (in)                     float64
Wind Gust (mph)               float64
Conditions                     object
dtype: object

In [5]:
# convert 'Date time' object into datetime format
df['Date time'] = pd.to_datetime(df['Date time'], infer_datetime_format=True)

In [6]:
# look up weather conditions
df['Conditions'].value_counts()

Clear                     206892
Partially cloudy          145829
Rain, Partially cloudy    108079
Rain, Overcast             58070
Overcast                   25485
Rain                       23756
Snow, Partially cloudy     18739
Snow, Overcast             17635
Snow                        5306
Name: Conditions, dtype: int64

In [7]:
# Aggregte conditions into 3 groups and label to numeric:
# 'Clear': 0; 'Partially cloudy', 'Overcast': 1; 'Rain, Partially cloudy', 'Rain, Overcast', 'Rain': 2;
# 'Snow, Partially cloudy', 'Snow, Overcast', 'Snow': 3
df['Conditions'] = df['Conditions'].replace({'Clear': 0, 'Partially cloudy': 1, 'Overcast': 1, 'Rain, Partially cloudy': 2, 'Rain, Overcast': 1,
                                            'Rain': 2, 'Snow, Partially cloudy': 3, 'Snow, Overcast': 3, 'Snow': 3})

In [9]:
# These are converted conditions
df['Conditions'].value_counts().sort_index()

0    206892
1    229384
2    131835
3     41680
Name: Conditions, dtype: int64

In [10]:
# create column year for future groupping
df['Year'] = df['Date time'].dt.year

In [11]:
grouped = df.groupby(['City', 'State', 'Year'], as_index=False)

In [13]:
# create a new data frame with conditions, city, state and year features
new_df = pd.DataFrame()
for year in grouped.groups:
    group = grouped.get_group(year)
    
    # create series with city, state, year columns
    series1 = pd.Series({'City':year[0], 'State': year[1], 'Year':year[2]})
    
    # get conditions per each city per each year
    series2 = pd.Series(dict(group['Conditions'].value_counts().sort_index()))
    # concat series
    concatenated = pd.Series(data=pd.concat([series1, series2]))
    # append df               
    new_df = new_df.append(concatenated, ignore_index=True)

In [14]:
new_df.head()

,0,1,2,3,City,State,Year
0,90.0,159.0,72.0,44.0,Akron,OH,2017.0
1,74.0,175.0,60.0,56.0,Akron,OH,2018.0
2,76.0,166.0,78.0,45.0,Akron,OH,2019.0
3,91.0,157.0,71.0,47.0,Akron,OH,2020.0
4,14.0,32.0,3.0,26.0,Akron,OH,2021.0


In [15]:
# rename conditions columns
new_df = new_df.rename(columns={0: "SunnyDays_avgPerYear", 1: "CloudyDays_avgPerYear", 2: 'RainyDays_avgPerYear', 3: 'SnowyDays_avgPerYear'})

In [16]:
new_df.head()

,SunnyDays_avgPerYear,CloudyDays_avgPerYear,RainyDays_avgPerYear,SnowyDays_avgPerYear,City,State,Year
0,90.0,159.0,72.0,44.0,Akron,OH,2017.0
1,74.0,175.0,60.0,56.0,Akron,OH,2018.0
2,76.0,166.0,78.0,45.0,Akron,OH,2019.0
3,91.0,157.0,71.0,47.0,Akron,OH,2020.0
4,14.0,32.0,3.0,26.0,Akron,OH,2021.0


In [18]:
# remove 2021 year because it's not a full year
new_df = new_df[~(new_df['Year'] == 2021)]

In [19]:
# check for nan values
new_df.isna().sum()

SunnyDays_avgPerYear       0
CloudyDays_avgPerYear      8
RainyDays_avgPerYear       0
SnowyDays_avgPerYear     571
City                       0
State                      0
Year                       0
dtype: int64

In [20]:
# nan values is where condition is not occured, therefore fill it with 0
new_df = new_df.fillna(0)

In [21]:
# create a new df with calculated mean 
conditions_df = pd.DataFrame()

# group cities
cities = new_df.groupby(['City', 'State'], as_index=False)

for city in cities.groups:
    group = cities.get_group(city)
    
    # get columns city, state
    series1 = pd.Series({'City': city[0], 'State': city[1]})
    
    # get the mean, round and convert float to int
    series2 = pd.Series(group.mean().round().astype('int64')[:4])
    
    # concatenate 2 series
    concatenated = pd.Series(data=pd.concat([series1, series2]))
    # append df
    conditions_df = conditions_df.append(concatenated, ignore_index=True)

In [22]:
conditions_df.head()

,City,CloudyDays_avgPerYear,RainyDays_avgPerYear,SnowyDays_avgPerYear,State,SunnyDays_avgPerYear
0,Akron,164.0,70.0,48.0,OH,83.0
1,Albany,126.0,91.0,0.0,GA,148.0
2,Albany,239.0,67.0,46.0,NY,13.0
3,Albany,102.0,129.0,6.0,OR,129.0
4,Albuquerque,242.0,44.0,1.0,NM,79.0


In [23]:
# looks like float was not converted to int
conditions_df[['CloudyDays_avgPerYear', 'RainyDays_avgPerYear', 'SnowyDays_avgPerYear', 'SunnyDays_avgPerYear']] = conditions_df[['CloudyDays_avgPerYear', 'RainyDays_avgPerYear', 'SnowyDays_avgPerYear', 'SunnyDays_avgPerYear']].astype('Int64')

In [24]:
conditions_df.head()

,City,CloudyDays_avgPerYear,RainyDays_avgPerYear,SnowyDays_avgPerYear,State,SunnyDays_avgPerYear
0,Akron,164,70,48,OH,83
1,Albany,126,91,0,GA,148
2,Albany,239,67,46,NY,13
3,Albany,102,129,6,OR,129
4,Albuquerque,242,44,1,NM,79


In [25]:
# save df to csv
conditions_df.to_csv('data/weather_conditions.csv', index=False)

In [30]:
final = pd.read_csv('../../datasets/datasets_to_merge/updated/final.csv', index_col=0)

In [31]:
final.head()

,City,State,TotalPop,Men,Women,Hispanic,White,Black,Native,Asian,Pacific,Income,IncomeErr,IncomePerCap,IncomePerCapErr,Poverty,ChildPoverty,Employed,Unemployment,PrivateWork,PublicWork,SelfEmployed,FamilyWork,Professional,Service,Office,Construction,Production,Drive,Carpool,Transit,Walk,OtherTransp,WorkAtHome,MeanCommute,Rent,Year,Population,Violent crime,Murder and nonnegligent manslaughter,Rape,Robbery,Aggravated assault,Property crime,Burglary,Larceny- theft,Motor vehicle theft,Arson,Crime Rate per 1000,Crime Rating,Days with AQI,Good Days,Moderate Days,Unhealthy for Sensitive Groups Days,Unhealthy Days,Very Unhealthy Days,Hazardous Days,Max AQI,90th Percentile AQI,Median AQI,Days CO,Days NO2,Days Ozone,Days SO2,Days PM2.5,Days PM10,Air Quality Index,Diversity Index,Nearest,lat,lon,SingleFamilyHousingAvgValue,CondoAvgValue,1-BedroomAvgValue,2-BedroomAvgValue,3-BedroomAvgValue,4-BedroomAvgValue,5+-BedroomAvgValue
index,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,Akron,OH,267782,129623,138086,1.9,67.4,23.2,0.2,3.6,0.0,3068406,504125,1732548,258991,19.6,26.2,126247,8.5,85.4,10.3,4.1,0.1,32.3,19.8,25.3,6.6,15.6,83.3,8.5,3.1,1.6,0.6,2.7,21.9,911,2020,197882,1782,27,181,328,1246,6568,1686,4305,577,65,42.20,High,274,203,68,3,0,0,0,107,63,43,0,0,132,0,142,0,Good,0.490241,"126,56,332,355,295",41.044852,-81.520048,84363.0,146090.0,54873.0,68915.0,90885.0,114421.0,115066.0
1,Albany,GA,93421,43571,49835,2.8,29.2,65.2,0.1,0.9,0.0,1025141,199233,547674,89500,29.6,41.9,35704,16.2,71.9,23.7,4.1,0.1,30.7,21.4,23.1,8.1,16.3,78.1,12.4,1.8,2.3,0.7,4.3,19.1,814,2020,74989,790,12,32,165,581,3452,729,2489,234,17,56.57,High,244,182,60,2,0,0,0,114,63,37,0,0,0,0,244,0,Good,0.488771,"162,372,139,351,249",31.567783,-84.161923,89713.0,104091.0,53131.0,31176.0,86154.0,194355.0,258224.0
2,Albany,NY,147794,70664,77096,7.8,60.3,20.1,0.1,7.6,0.0,1917135,331452,1102265,158632,17.4,17.8,74293,6.6,71.6,24.3,3.9,0.0,42.9,20.5,25.2,4.6,6.6,70.4,7.2,10.1,7.3,1.7,2.9,18.9,1205,2020,97221,736,4,60,189,483,2919,445,2315,159,16,37.59,Medium,274,240,34,0,0,0,0,75,53,39,1,0,169,0,104,0,Good,0.584133,"319,358,336,145,79",42.652580,-73.756633,227082.0,154120.0,149080.0,186585.0,223518.0,265114.0,308318.0
3,Albany,OR,61555,30378,31168,11.4,82.7,0.6,0.7,1.9,0.1,542299,75753,268177,35254,14.3,18.6,27721,8.2,78.5,15.9,5.4,0.1,33.3,18.4,22.7,9.9,15.5,79.0,10.7,0.3,2.6,2.0,5.1,20.8,1197,2020,54993,70,1,10,16,43,1467,128,1242,97,17,27.95,Medium,182,161,21,0,0,0,0,70,51,19,0,0,0,0,182,0,Good,0.302632,"29,132,230,383,202",44.627722,-123.094409,327516.0,222806.0,230224.0,248308.0,317989.0,389785.0,444497.0
4,Albuquerque,NM,632240,309415,322758,50.4,38.5,2.5,4.0,2.4,0.1,7693803,1270581,4099669,581026,18.2,24.8,292907,6.8,74.5,19.6,5.6,0.2,39.8,20.0,24.0,8.3,7.7,80.2,9.1,1.8,1.8,2.5,4.3,21.7,1013,2020,561920,7596,84,486,1699,5327,26059,0,20634,5425,98,59.89,High,183,101,82,0,0,0,0,100,65,50,1,0,127,0,25,30,Good,0.594958,"359,258,386,360,127",35.106766,-106.629181,245801.0,177085.0,154993.0,186294.0,237037.0,315793.0,391279.0


In [32]:
merged = pd.merge(final, conditions_df, how="left", on=["City", "State"],  left_index=False,  right_index=False)

In [33]:
merged.head()

,City,State,TotalPop,Men,Women,Hispanic,White,Black,Native,Asian,Pacific,Income,IncomeErr,IncomePerCap,IncomePerCapErr,Poverty,ChildPoverty,Employed,Unemployment,PrivateWork,PublicWork,SelfEmployed,FamilyWork,Professional,Service,Office,Construction,Production,Drive,Carpool,Transit,Walk,OtherTransp,WorkAtHome,MeanCommute,Rent,Year,Population,Violent crime,Murder and nonnegligent manslaughter,Rape,Robbery,Aggravated assault,Property crime,Burglary,Larceny- theft,Motor vehicle theft,Arson,Crime Rate per 1000,Crime Rating,Days with AQI,Good Days,Moderate Days,Unhealthy for Sensitive Groups Days,Unhealthy Days,Very Unhealthy Days,Hazardous Days,Max AQI,90th Percentile AQI,Median AQI,Days CO,Days NO2,Days Ozone,Days SO2,Days PM2.5,Days PM10,Air Quality Index,Diversity Index,Nearest,lat,lon,SingleFamilyHousingAvgValue,CondoAvgValue,1-BedroomAvgValue,2-BedroomAvgValue,3-BedroomAvgValue,4-BedroomAvgValue,5+-BedroomAvgValue,CloudyDays_avgPerYear,RainyDays_avgPerYear,SnowyDays_avgPerYear,SunnyDays_avgPerYear
0,Akron,OH,267782,129623,138086,1.9,67.4,23.2,0.2,3.6,0.0,3068406,504125,1732548,258991,19.6,26.2,126247,8.5,85.4,10.3,4.1,0.1,32.3,19.8,25.3,6.6,15.6,83.3,8.5,3.1,1.6,0.6,2.7,21.9,911,2020,197882,1782,27,181,328,1246,6568,1686,4305,577,65,42.20,High,274,203,68,3,0,0,0,107,63,43,0,0,132,0,142,0,Good,0.490241,"126,56,332,355,295",41.044852,-81.520048,84363.0,146090.0,54873.0,68915.0,90885.0,114421.0,115066.0,164,70,48,83
1,Albany,GA,93421,43571,49835,2.8,29.2,65.2,0.1,0.9,0.0,1025141,199233,547674,89500,29.6,41.9,35704,16.2,71.9,23.7,4.1,0.1,30.7,21.4,23.1,8.1,16.3,78.1,12.4,1.8,2.3,0.7,4.3,19.1,814,2020,74989,790,12,32,165,581,3452,729,2489,234,17,56.57,High,244,182,60,2,0,0,0,114,63,37,0,0,0,0,244,0,Good,0.488771,"162,372,139,351,249",31.567783,-84.161923,89713.0,104091.0,53131.0,31176.0,86154.0,194355.0,258224.0,126,91,0,148
2,Albany,NY,147794,70664,77096,7.8,60.3,20.1,0.1,7.6,0.0,1917135,331452,1102265,158632,17.4,17.8,74293,6.6,71.6,24.3,3.9,0.0,42.9,20.5,25.2,4.6,6.6,70.4,7.2,10.1,7.3,1.7,2.9,18.9,1205,2020,97221,736,4,60,189,483,2919,445,2315,159,16,37.59,Medium,274,240,34,0,0,0,0,75,53,39,1,0,169,0,104,0,Good,0.584133,"319,358,336,145,79",42.652580,-73.756633,227082.0,154120.0,149080.0,186585.0,223518.0,265114.0,308318.0,239,67,46,13
3,Albany,OR,61555,30378,31168,11.4,82.7,0.6,0.7,1.9,0.1,542299,75753,268177,35254,14.3,18.6,27721,8.2,78.5,15.9,5.4,0.1,33.3,18.4,22.7,9.9,15.5,79.0,10.7,0.3,2.6,2.0,5.1,20.8,1197,2020,54993,70,1,10,16,43,1467,128,1242,97,17,27.95,Medium,182,161,21,0,0,0,0,70,51,19,0,0,0,0,182,0,Good,0.302632,"29,132,230,383,202",44.627722,-123.094409,327516.0,222806.0,230224.0,248308.0,317989.0,389785.0,444497.0,102,129,6,129
4,Albuquerque,NM,632240,309415,322758,50.4,38.5,2.5,4.0,2.4,0.1,7693803,1270581,4099669,581026,18.2,24.8,292907,6.8,74.5,19.6,5.6,0.2,39.8,20.0,24.0,8.3,7.7,80.2,9.1,1.8,1.8,2.5,4.3,21.7,1013,2020,561920,7596,84,486,1699,5327,26059,0,20634,5425,98,59.89,High,183,101,82,0,0,0,0,100,65,50,1,0,127,0,25,30,Good,0.594958,"359,258,386,360,127",35.106766,-106.629181,245801.0,177085.0,154993.0,186294.0,237037.0,315793.0,391279.0,242,44,1,79


In [35]:
import sqlalchemy
sqlalchemy.__version__

'1.4.7'

In [36]:
from dotenv import load_dotenv

In [37]:
%load_ext dotenv
%dotenv
de = %env

In [38]:
# Initate SQLalchemy
engine = sqlalchemy.create_engine(de['DATABASE_URL'])
conneciton = engine.connect()

In [39]:
# Add combined data into Database
merged.to_sql('data',con=engine,method='multi', if_exists='replace')